In [1]:
import pandas as pd
import spacy 
nlp = spacy.load('en')

In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter 
from bs4 import BeautifulSoup 
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")
stop = set(stopwords.words('english'))
import string

import pickle

def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler,protocol=2)
    filehandler.close()

def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

### Load Data

In [ ]:
# # READS THE EXCEL FILE
# # INSTEAD OF FILE, WE SHOULD BE PASSING DATAFRAMES (FOR THE SELECTION OF SOURCES)
# conversations = pd.read_excel('Coke vs Pepsi Twitter Data for NLP 1.xlsx')
# # DROPS THE EMPTY OBSERVATIONS. ARE WE SUPPOSED TO DO THAT IN THIS CASE? THERE ARE NO BLANK TEXTS IN THIS SET
# conversations = conversations.dropna(subset=['Sound Bite Text'])

In [4]:
# LOADS CSV WITH SELECTED DATA
df = pd.read_csv('conversation_tagged.csv')
conversations = df['Sound Bite Text']

#### Function to Clean Text used later

In [76]:
# CLEANS TEXT BY REPLACING CHARACTERS
def getCleanText(texts, lower = True):
    theList=[]
    for text in texts:
        text = str(text)
        parse_text = BeautifulSoup(text).get_text()
        letters_only = re.sub(r'http[s]?:\/\/(?:[a-zA-Z]|[0-9]|[$-@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' url ', 
                              parse_text, flags=re.MULTILINE)
        letters_only = re.sub('^(1?[0-9]|2[0-3]):[0-5][0-9]$','time',letters_only)
        letters_only = re.sub('(name|NAME)\d{1,}','username',letters_only)
        letters_only = re.sub("[^a-zA-Z0-9\.:']",  
                          " ",                   
                          letters_only)
        letters_only = letters_only.replace('\n'," ")
        letters_only = letters_only.replace('\r'," ")
        letters_only = re.sub('[.]{2,}', '. ', letters_only)
        if lower:
            letters_only = letters_only.lower()
#         letters_only = letters_only.replace("  ","|")
#         letters_only = letters_only.replace(" ","_")
#         letters_only = letters_only.replace("|"," ")
#         letters_only = letters_only.strip("_")
#        print(letters_only)
        words = CountVectorizer(stop_words='english').build_tokenizer()(letters_only)
        meaningful_words = [ w for w in words if len(w)> 0 and len(w)<20]
        clean_text = " ".join(meaningful_words)
        theList.append(clean_text)
    finalList = "_".join(theList)
    return finalList

In [6]:
#define some parameters  
noisy_pos_tags = ['PROP'] # TAGGING OF PROP
min_token_length = 2 # TWO CHARACTERS OR TWO TOKENS?

#Function to check if the token is a noise or not  
def isNoise(token):     
    is_noise = False
    if token.pos_ in noisy_pos_tags: # FROM LIST ABOVE
        is_noise = True 
    elif token.is_stop == True: # FROM STOP WORDS LIST 
        is_noise = True
    elif len(token.string) <= min_token_length: # FROM PARAMETER ABOVE
        is_noise = True
    return is_noise 
def cleanup(token, lower = True):
    if lower: # TURNS ALL TEXT TO LOWERCASE
        token = token.lower()
    return token.strip() # REMOVES SPACES FROM THE BEGINNING AND END

### Named Entity Recognition

In [80]:
def get_entities(sent):
    try:
        document = nlp(sent)
    except:
        return ["error"]
    
    entities_list = []
    labels = set([w.label_ for w in document.ents]) 
    for label in labels: 
        entities = [cleanup(e.string, lower=False) for e in document.ents if label==e.label_] 
        entities = list(set(entities))
        if label == "ORG": # PICKS ORG ENTITIES
            entities_list.extend(entities)
    if len(entities_list) > 0:
        return entities_list
    else:
        return ["<UNK>"]
org_list = map(get_entities, conversations)

#### Concatentate Named Entities together from all rows

In [81]:
all_orgs_clean = map(getCleanText, org_list)
entities_per_text = list(all_orgs_clean)
#print(list(entities_per_text))

/Users/erikdaily/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/erikdaily/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


elise amendola
file ap
the associated press
mobile   apps file
pepsi
3q
pepsico inc.
file ap photo
n.y. pepsico inc.
elise amendola
file ap
the associated press
mobile   apps file
pepsi
3q
pepsico inc.
file ap photo
n.y. pepsico inc.
stellar novellas
coca cola
  executive creative director ashley sword
chrysler
stellar scholars
stellar romance
roundtable
the district of columbia
mcdonald
crisps
coca cola
walkers
coca cola
instagram
coca cola
nra
coca cola
nra
coca cola company
coca cola
coca
coca cola
coca
coca cola
coca
coca
coca
albertsons
sheet pan nachos recipe
coca
coca
randalls
coca cola
coca cola
coca cola
monsanto
second equifax
third equifax
pepsi
costco
smackdown tag team champions
cell
tye dillinger
wwe
live
pepsi
pepsi

pepsi
slam dunk quarter
pepsi
pepsico inc.
pep
frito lay north america
quaker foods north america
pepsi
pepsico inc. pepsico inc.
tropicana
frito lay
frito lay north america
quaker
quaker foods north america
pepsico company profile pepsico
company
pepsi cola


the kardashians
pepsi
pepsi
related
sugar
coca cola
ttab
coke
zero
the twin cities
pepsi
related
sugar
coca cola
ttab
coke
zero
the twin cities
pepsi
sugar
coca cola
ttab
coke
zero
etf
procter   gamble co pg
coca cola co
kxi
margin expansion
fcc
first equifax
second equifax
wells fargo
pepsi
nasdaq
third equifax hearingfriday   u.s.
click
youth empowered nigeria
youth empowered
the coca cola company
nbc
the nigerian bottling company limited
coca cola hellenic bottling company
port harcourt
youth empowered nigeria
youth empowered
the coca cola company
nbc
the nigerian bottling company limited
coca cola hellenic bottling company
port harcourt
coke
 s



diet pepsi
vegas
the denver post
mdt
the denver post posted
avalanche
the coca cola company
pepsi
previous kendall jenner
the coca cola company
coca cola
eu
coca cola
gocompare
amazon
coca cola
the georgia bulldogs

coca cola
pepsi
email
coca cola
arca continental
wfaa
topo chico mineral water
rohan oza
coca cola
coca cola
topo chico min

diet
coca cola
jonesboro  paragould
central
the missouri bootheel
the coca cola company
21st century beverage
coca cola
jonesboro  paragould
central
the missouri bootheel
the coca cola company
21st century beverage
coca cola
jonesboro  paragould
central
the missouri bootheel
the coca cola company
21st century beverage
coca cola
jonesboro  paragould
21st century beverage
the coca cola company
coca cola
21st century beverage
jonesboro
the coca cola company
coca cola
tesco
coca cola
corinth coca cola
jonesboro
the coca cola company
coca cola
corinth coca cola
jonesboro
avon kenneth weaver
corinth
bbc
mtn
kuwtk
pepsi
the shade room

coca cola north america
coke
coca cola's
venturing   emerging brands
coca cola acquires
csp daily news
coca cola north america
coke
coca cola's
venturing   emerging brands
coca cola acquires
csp daily news
cummings 
coca cola
madame sirleaf s
coca cola north america
coke
coca cola's
venturing   emerging brands
coca cola acquires
csp daily news
coca cola
the cer

coca cola
coca cola co.
coca cola
coca cola co.
coca cola
coca cola co.
coca cola
pepsi
pepsi
pepsi
pepsi
coca cola
coca cola
mblm
miller brewing
coca cola
mblm
miller brewing
coca cola
mblm
miller brewing
 s biq
biq
pepsi
tropicana  
baskin robbins
coca cola
quaker oats
coca cola
bell telephone
atari
rca corp
koss
pan am
tsingtao
pepsi
cuisinart
coca cola
conair
gentle readers
coca cola

coca cola
subhaanallah
south hampton resources inc.
the private securities litigation reform act
gevo
local vegas tv
coca cola
nbc s ksnv
abc

pepsi
chili's
coke
mexico earthquake aid mexico news
coca cola company
bbva bancomer   grupo bbva
walmart m xico
el economista
ford motor company
coca cola company
samsung
apple inc.
at t
scotiabank
walmart m xico
el economista
google
bbva bancomer
scottish daily record
holly
coke
permissionmetro holly willoughby
scottish daily record raspberry
lufthansa
coca cola company

suv
ginisun
nigeria news
the voice
daily times
coca cola
idyl
voice
dubaisun
airtel niger

...

coca cola company
adobe photoshop
coreldraw
coke
adobe illustrator
coke
coca cola company
coke
coca cola company
coca cola
cadbury
bhs
colgate
poundworld
share comments
coca cola
uk equity
costco
gillette 
american express
heineken
berkshire hathaway
unilever
geico
dutch
elise amendola
ap
frito lay
ap photo
pepsico
sports member center
pepsi
louisiana news
pepsico inc.
9 news
coke
sugar
pearson
fda
pepsi
mylan
slam dunk quarter ford
connectivity or autonomous
mizuho securities usa llc
pepsi center
elise amendola
ap
ap photo
frito lay
pepsico
pepsi
pepsico inc.
al member center
pepsico
pepsi
gatorade
lifewtr
pepsico
pepsi
pepsico
pepsi
gatorade
lifewtr
pepsico
pepsi
the schools recycling awards
tlhatlogang secondary school
the schools recycling programme
coca cola beverages sa
the sheds
schools recycling awards
va
elise amendola
ap
ap photo
frito lay
pepsico
pepsi
pepsico inc.
kellogg
coca cola and ab
nature
coca cola
coca cola americas
nature
cahillane
coca cola enterprises
coca cola a

 s
coca cola
pepsi
 s
elise amendola
ap
ap photo
frito lay
pepsico
sports member center
pepsi
pepsico inc.
purchase
mason city news
pepsico inc's
pepsi
pepsico
pepsico inc's
pepsi
pepsico
frito lay
pepsico
coca cola co
cheetos
nooyi
pepsico
coca cola company
sruthi ramakrishnan
morgan stanley
rolex
nasa
rolex gmt master
pepsi
gp
gear patrol
pepsi
compucom
pepsi
coca cola bottling company
ceo
coca cola bottling
valparaiso sales center coca
united
okaloosa
coca cola united
coca cola
coca cola bottling company
ceo
coca cola bottling
valparaiso sales center coca
united
okaloosa
coca cola united
coca cola
coca cola bottling company
ceo
united
coca cola united
coca cola
coca cola bottling company
coca cola  day. 
coca cola
coca cola  day. 
coca cola
coca cola  day. 
coca cola
coke
coca cola  day. 
coca cola
coke
coca cola
coke
pepsi
facebook
coke
pepsi
ap
compucom
pepsi
compucom
usa today sports florida
usa today sports
the boston bruins
maxim mamin
colorado avalanche
usa today sports
usa to

coca cola
checkers
ex springbok
mathebula
toyota
coca cola
coca cola company
strategic security
strategic security   aviation
hush
coca cola co.
coca cola
coca cola company
strategic security
strategic security's
strategic security   aviation
hush
coca cola co.
coca cola
strategic security
strategic security's
strategic security   aviation
hush
vice president
coca cola
strategic security
cso
exelon corp.
hush's
north american transmission forum
coca cola
coca cola
nike
anderson printing house
apple
elise amendola
ap
frito lay
pepsico
pepsi
ap image
pepsico inc.
elise amendola
ap
frito lay
pepsico
pepsi
ap image
pepsico inc.
gatorade and
pepsi
frito lay
coke
pepsi
coca cola
rogers  freedom mobile
comcast
bonke
radicalmedia
vfx
cabin editing company
red cross
nike
ogilvy   mather
alkemy x creative
alkemy x
crown
nascar
coca cola
charlotte motor speedway
dillon s chevy
drew gulak
live review
live
the raw cruiserweight division
the wwe network
ariya daivari
enzo
pepsi
versace
the daisy duk

file ap photo
mountain dew
pepsico
pepsi
mountain dew
interpublic group
homegoods
coca cola
bbdo
chevrolet
mgm resorts
mccann worldgroup s
coca cola
sugar
coke zero sugar
coke zero 
coca cola
sugar
coke zero sugar
coke zero 
coca cola
tripadvisor
caterpillar
un
bds
black
pepsi
nfl
media
yellow


elise amendola
file ap
the associated press
mobile   apps file
pepsi
3q
pepsico inc.
file ap photo
n.y. pepsico inc.
elise amendola
file ap
the associated press
mobile   apps file
pepsi
3q
pepsico inc.
file ap photo
n.y. pepsico inc.
elise amendola
file ap
the associated press
pepsi
3q
pepsico inc.
file ap photo
n.y. pepsico inc.
elise amendola
file ap
the associated press
pepsi
3q
pepsico inc.
file ap photo
n.y. pepsico inc.
pepsi
principal at prevalent equity
dasani
the coca cola company
  viral bhayani
pepsi   whole foods
instacart
the next pedestal for entrepreneurs
instacart s
grand prix senior
pepsi
suzuki s vintage parts programme
lucky strike
suzuki
lucky strike 
telefonica movistar rgv

pepsi
martine cajucom
instagram story





pepsico
pepsi
mountain dew
brent lewis
avalanche
the denver post
taco bell

the pepsi center
the denver post
malone
the denver nuggets'
nuggets


#### Count of Named Entitites

In [82]:
entities_dict = Counter(entities_per_text)
entities_list_sorted = sorted(entities_dict, key = entities_dict.get, reverse=True)
entities_list_sorted_tup = [(entity,entities_dict[entity]) for entity in entities_list_sorted]
entities_list_sorted_tup

[('', 1976),
 ('pepsi', 1494),
 ('coca cola', 1083),
 ('coke', 427),
 ('coke_coca cola', 202),
 ('coca cola_coke', 168),
 ('coke_pepsi', 129),
 ('coca cola_the georgia bulldogs', 98),
 ('pepsi_coke', 94),
 ('fortune_pepsi_pepsi ceo indra', 81),
 ('coca', 78),
 ('coca cola_topo chico mineral water_wfaa_tccc_arca continental', 64),
 ('pepsi_reebok_nike_the milwaukee brewers_major league baseball_food network',
  60),
 ('mcdonald_coca cola_converse_nba', 60),
 ('target_converse_pepsi_hiv_los angeles lakers_nfl_kfc_nestle', 60),
 ('target_converse_the american beef industry council_pepsi_hiv_kfc_nestle',
  60),
 ('coca cola_education review young_delta state_advan_ndubuzo ebube fortune_jet stanley madu chika nwobbi_the advertisers association of',
  43),
 ('pepsico_pepsi_mountain dew', 41),
 ('coke_dc_pepsi of spandex_marvel', 38),
 ('the coca cola company', 37),
 ('pepsico_pepsi', 35),
 ('coca cola_ap photo_the nascar hall of fame_nascar_pocono raceway', 34),
 ('coca cola_national univers

In [85]:
df = df.assign(entities = entities_per_text)
print(df.shape)
print(len(entities_per_text))

(18340, 17)
18340


In [84]:
dump(df, "df_sel_entities.pkl")
dump(entities_list_sorted_tup, "df__sel_entities_count.pkl")
dump(entities_per_text, "df_sel_entities_list.pkl")